In [1]:
#importing nesessary libraries
import pandas as pd
import matplotlib as plt
import numpy as np
import math

In [2]:
#reading input dataset from csv to dataframe
df = pd.read_csv('monthly-production.csv')
df[['RPT_DATE','Dt_Treat']] = df[['RPT_DATE','Dt_Treat']].astype('datetime64')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 291495 entries, 0 to 291494
Data columns (total 12 columns):
 #   Column         Non-Null Count   Dtype         
---  ------         --------------   -----         
 0   API            291495 non-null  int64         
 1   Pool_Nm        291495 non-null  object        
 2   RPT_DATE       291495 non-null  datetime64[ns]
 3   DAYS_PROD      291495 non-null  int64         
 4   BBLS_OIL_COND  291495 non-null  int64         
 5   OIL_RUNS       291495 non-null  int64         
 6   BBLS_WTR       291495 non-null  int64         
 7   MCF_GAS        291495 non-null  int64         
 8   MCF_SOLD       291495 non-null  int64         
 9   FLARED         291495 non-null  int64         
 10  VENTED         291495 non-null  int64         
 11  Dt_Treat       291495 non-null  datetime64[ns]
dtypes: datetime64[ns](2), int64(9), object(1)
memory usage: 26.7+ MB


In [3]:
df.keys()

Index(['API', 'Pool_Nm', 'RPT_DATE', 'DAYS_PROD', 'BBLS_OIL_COND', 'OIL_RUNS',
       'BBLS_WTR', 'MCF_GAS', 'MCF_SOLD', 'FLARED', 'VENTED', 'Dt_Treat'],
      dtype='object')

In [4]:
df

,API,Pool_Nm,RPT_DATE,DAYS_PROD,BBLS_OIL_COND,OIL_RUNS,BBLS_WTR,MCF_GAS,MCF_SOLD,FLARED,VENTED,Dt_Treat
0,33007001030000,MADISON,2009-01-01,31,705,704,5206,51,24,0,0,2010-12-16
1,33007001030000,MADISON,2009-02-01,28,597,597,4603,46,25,0,0,2010-12-16
2,33007001030000,MADISON,2009-03-01,31,701,704,5014,61,33,0,0,2010-12-16
3,33007001030000,MADISON,2009-04-01,30,690,687,4853,65,37,0,0,2010-12-16
4,33007001030000,MADISON,2009-05-01,31,734,736,5116,58,32,0,0,2010-12-16
...,...,...,...,...,...,...,...,...,...,...,...,...
291490,33105036600000,BAKKEN,2015-02-01,28,10270,10197,9011,11216,0,11216,0,2014-11-19
291491,33105036750000,BAKKEN,2014-11-01,0,0,0,0,0,0,0,0,2014-11-11
291492,33105036750000,BAKKEN,2014-12-01,23,5690,5231,26625,8492,0,8492,0,2014-11-11
291493,33105036750000,BAKKEN,2015-01-01,27,9275,9602,14695,11213,0,11213,0,2014-11-11


In [5]:
#Number of wells in the dataset
api_unique_list=df['API'].unique()
len(api_unique_list)

8939

In [7]:
#Creating average volume of fluids produced each days for each month of production record
df_ind=df.set_index(['API','RPT_DATE'])
#df_trim=df_ind
df_ind['OIL_per_day'] = df_ind['OIL_RUNS']/df_ind['DAYS_PROD']
df_ind['GAS_per_day'] = df_ind['MCF_GAS']/df_ind['DAYS_PROD']
df_ind['WATER_per_day'] = df_ind['BBLS_WTR']/df_ind['DAYS_PROD']
df_trim=df_ind[['DAYS_PROD','OIL_per_day', 'GAS_per_day', 'WATER_per_day']]
df_trim = df_trim.fillna(0)
df_trim

DAYS_PROD  OIL_per_day  GAS_per_day  WATER_per_day
API            RPT_DATE                                                      
33007001030000 2009-01-01         31    22.709677     1.645161     167.935484
               2009-02-01         28    21.321429     1.642857     164.392857
               2009-03-01         31    22.709677     1.967742     161.741935
               2009-04-01         30    22.900000     2.166667     161.766667
               2009-05-01         31    23.741935     1.870968     165.032258
...                              ...          ...          ...            ...
33105036600000 2015-02-01         28   364.178571   400.571429     321.821429
33105036750000 2014-11-01          0     0.000000     0.000000       0.000000
               2014-12-01         23   227.434783   369.217391    1157.608696
               2015-01-01         27   355.629630   415.296296     544.259259
               2015-02-01         13   432.692308   303.000000     623.615385

[291495 rows x 4 columns]

In [8]:
#function to extract production metric per well id for a specific period
def get_production_for_period(df, period):
    
    time = 0
    cum_water=0
    cum_gas=0
    cum_oil=0
    
    for row in df.iterrows():

        days=row[1][0]
        if time < period:
            if time+days <= period:
                time=time+days
                cum_water=cum_water+row[1][3]*days
                cum_gas=cum_gas+row[1][2]*days
                cum_oil=cum_oil+row[1][1]*days
            else:
                days_left = period-time
                time = time+days_left
                cum_water=cum_water+row[1][3]*days_left
                cum_gas=cum_gas+row[1][2]*days_left
                cum_oil=cum_oil+row[1][1]*days_left
                break
    
    if time >= period:

        cum_water=round(cum_water/10,1)*10
        cum_oil=round(cum_oil/10,1)*10
        cum_wc=round(cum_water/(cum_water+cum_oil)*100,3) # water cut
        cum_gor=round(cum_gas/cum_oil*1000,3) # gas-oil-ratio
        
        return cum_water, cum_gor, cum_wc, cum_oil
        
#test_df=df_trim.loc[api]
#data = get_production_for_period(test_df, int(per))
#print(data)

In [8]:
#per= '30'  # '30', '90' '180', '365', '545', '730'
#api=33007011110000
#df_out[df_out['api']==api][['cum_water_'+per,'cum_gor_'+per,'cum_wc_'+per,'cum_oil_'+per]]

In [9]:
#function to extract production metric per well id
def get_production_metrics(api, df):
    
    ser_list=[]
    
    for i in [30, 90, 180, 365, 545, 730]:
        data = get_production_for_period(df, i)
        ser = pd.Series(data , index =['cum_water_'+str(i),'cum_gor_'+str(i), 'cum_wc_'+str(i), 'cum_oil_'+str(i)])
        ser_list.append(ser)
        
    ser_row = pd.concat(ser_list,axis=0)
    ser_row['api']=api
    ser_row=ser_row.sort_index()
    #print(ser_row)
    return ser_row
    
    #ser_30 = pd.Series(data_30, index =['cum_water_30','cum_gor_30', 'cum_wc_30', 'cum_oil_30']) 
    #return 
    
#df_test = df_trim.loc[api]
#data = get_production_metrics(api, df_test)
#print(data)

In [12]:
%%time

#run function to extract production metrics for all wells
data_list=[]

for api in api_unique_list:

    data_list.append(get_production_metrics(api, df_trim.loc[api]))
    
data_full = pd.concat(data_list,axis=1)

C:\Users\olyab\Anaconda3\lib\site-packages\ipykernel_launcher.py:17: RuntimeWarning: invalid value encountered in double_scalars
C:\Users\olyab\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  
C:\Users\olyab\Anaconda3\lib\site-packages\ipykernel_launcher.py:31: RuntimeWarning: divide by zero encountered in double_scalars
C:\Users\olyab\Anaconda3\lib\site-packages\ipykernel_launcher.py:16: RuntimeWarning: invalid value encountered in double_scalars
  app.launch_new_instance()
C:\Users\olyab\Anaconda3\lib\site-packages\ipykernel_launcher.py:31: RuntimeWarning: invalid value encountered in double_scalars
C:\Users\olyab\Anaconda3\lib\site-packages\ipykernel_launcher.py:30: RuntimeWarning: invalid value encountered in double_scalars
C:\Users\olyab\Anaconda3\lib\site-packages\ipykernel_launcher.py:15: RuntimeWarning: inva

Wall time: 4min 24s


In [13]:
data_final = data_full.transpose()
data_final['api']=data_final['api'].astype('int64')
#data_final.to_csv('DataWrangling_prod-by-operated-day.csv',index=False)
#data_final.style.format({'cum_gor_': '{0:.3f}'})
#data_final.applymap("{0:.2f}".format)

In [14]:
df_trim

DAYS_PROD  OIL_per_day  GAS_per_day  WATER_per_day
API            RPT_DATE                                                      
33007001030000 2009-01-01         31    22.709677     1.645161     167.935484
               2009-02-01         28    21.321429     1.642857     164.392857
               2009-03-01         31    22.709677     1.967742     161.741935
               2009-04-01         30    22.900000     2.166667     161.766667
               2009-05-01         31    23.741935     1.870968     165.032258
...                              ...          ...          ...            ...
33105036600000 2015-02-01         28   364.178571   400.571429     321.821429
33105036750000 2014-11-01          0     0.000000     0.000000       0.000000
               2014-12-01         23   227.434783   369.217391    1157.608696
               2015-01-01         27   355.629630   415.296296     544.259259
               2015-02-01         13   432.692308   303.000000     623.615385

[291495 rows x 4 columns]